# Mining and Classifiers
We used the random forest, decision tree, and naive bayes classifiers to classify our data.

Unfortunately, after many attempts we were unable to get random forest to work with our dataset. 

In [9]:
import importlib
import utils 
importlib.reload(utils)
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils
import plot_utils
importlib.reload(plot_utils)
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 
import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor, MyNaiveBayesClassifier, MyRandomClassifier, MyZeroRClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier
from tabulate import tabulate

music_table = MyPyTable()
music_fname = os.path.join("input_data", "music-data.csv")
music_table.load_from_file(music_fname)

music_data = music_table.data.copy()
del_list = [0, 0, 2]
for j in music_data:
    for i in del_list:
        del j[i]

# Discretizes artist and track popularity
myutils.discretize_popularity(music_data, 0)
myutils.discretize_popularity(music_data, 2)

# Discretizes duration
duration_col = myutils.get_column_by_index(music_data, 1)
cutoffs = myutils.compute_equal_width_cutoffs(duration_col, 3)
for i, value in enumerate(music_data):
    if value[1] <= 227131.33:
        value[1] = "short"
    elif value[1] > 227131.33 and value[1] <= 413062.67:
        value[1] = "medium"
    elif value[1] > 413062.67 and value[1] <= 598994:
        value[1] = "long"

# Discretizes available markets
markets_col = myutils.get_column_by_index(music_data, 4)
cutoffs = myutils.compute_equal_width_cutoffs(markets_col, 3)
for i, value in enumerate(music_data):
    if value[4] <= 60.0:
        value[4] = "small"
    elif value[4] > 60.0 and value[4] <= 119.0:
        value[4] = "medium"
    elif value[4] > 119.0:
        value[4] = "large"

# Discretizes genres
myutils.combine_genre(music_data)

# Discretizes danceability
danceability_col = myutils.get_column_by_index(music_data, 5)
cutoffs = myutils.compute_equal_width_cutoffs(danceability_col, 3)
for i, value in enumerate(music_data):
    if value[5] <= 0.32:
        value[5] = "1"
    elif value[5] > 0.32 and value[5] <= 0.65:
        value[5] = "2"
    elif value[5] > 0.65:
        value[5] = "3"

# Discretizes acousticness
acousticness_col = myutils.get_column_by_index(music_data, 6)
cutoffs = myutils.compute_equal_width_cutoffs(acousticness_col, 3)
for i, value in enumerate(music_data):
    if value[6] <= 0.33:
        value[6] = "1"
    elif value[6] > 0.33 and value[6] <= 0.66:
        value[6] = "2"
    elif value[6] > 0.66:
        value[6] = "3"

# Discretizes tempo
tempo_col = myutils.get_column_by_index(music_data, 7)
cutoffs = myutils.compute_equal_width_cutoffs(tempo_col, 3)
for i, value in enumerate(music_data):
    if value[7] <= 71.34:
        value[7] = "1"
    elif value[7] > 71.34 and value[7] <= 142.68:
        value[7] = "2"
    elif value[7] > 142.68:
        value[7] = "3"

m_labels, m_table = myutils.get_train_labels(music_data, 0)
label_list = []
for val in m_labels:
    if val not in label_list:
        label_list.append(val)
X_train, X_test, y_train, y_test = myevaluation.train_test_split(m_table, m_labels)
label_list.sort()

naive = MyNaiveBayesClassifier()
naive.fit(X_train, y_train)
y_pred_list = []
for val in X_test:
    y_pred_list.append(naive.predict(val))
matrix = myevaluation.confusion_matrix(list(y_test), y_pred_list, label_list)
for i, val in enumerate(matrix):
    val.insert(0, label_list[i])
print("===========================================")
print("Music Data - Naive Bayes Results")
print("===========================================")
print(tabulate(matrix, headers = label_list))
accuracy = myutils.compute_accuracy(matrix)
print("Accuracy = ", accuracy)
error_rate = myutils.compute_error_rate(matrix)
print("Error Rate = ", error_rate)

dec_tree = MyDecisionTreeClassifier()
dec_tree.fit(X_train, y_train)
y_pred_list = dec_tree.predict(X_test)

matrix = myevaluation.confusion_matrix(list(y_test), y_pred_list, label_list)
for i, val in enumerate(matrix):
    val.insert(0, label_list[i])

print("===========================================")
print("Music Data - Decision Tree Results")
print("===========================================")
print(tabulate(matrix, headers = label_list))
accuracy = myutils.compute_accuracy(matrix)
print("Accuracy = ", accuracy)
error_rate = myutils.compute_error_rate(matrix)
print("Error Rate = ", error_rate)



random_forest = MyRandomForestClassifier()
random_forest.fit(X_train, y_train, music_table.column_names, label_list)
pred_list = random_forest.predict(X_test)
# matrix = myevaluation.confusion_matrix(list(y_test), pred_list, label_list)
# for i, val in enumerate(matrix):
#     val.insert(0, label_list[i])
#     print("===========================================")
# print("Music Data - Random Forest Results")
# print("===========================================")
# print(tabulate(matrix, headers = label_list))
# accuracy = myutils.compute_accuracy(matrix)
# print("Accuracy = ", accuracy)
# error_rate = myutils.compute_error_rate(matrix)
# print("Error Rate = ", error_rate)





Music Data - Naive Bayes Results
                 Above Average    Average    Below Average    High    Low
-------------  ---------------  ---------  ---------------  ------  -----
Above Average               87          1                0     301      0
Average                      4          0                0      26      0
Below Average                0          0                0       4      0
High                        10          0                0      43      0
Low                         15          0                0      58      0
Accuracy =  0.23679417122040072
Error Rate =  0.07468123861566485
Music Data - Decision Tree Results
                 Above Average    Average    Below Average    High    Low
-------------  ---------------  ---------  ---------------  ------  -----
Above Average              385          0                0       0      4
Average                     28          1                0       0      1
Below Average                4          0           